In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt


#import geopandas as gpd

# Based on geopandas
#gdf = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))


### Speeches COP28-COP19:
1. Importing of document names (new dataframe - COP28_df)
2. Overview of available speeches per country
3. Overview of available languages per country

##### Importing dataframes of document names
Evaluating total number of speeches per cop and in total.

In [ ]:
cop_directories = ['COP28', 'COP27', 'COP26', 'COP25','COP24','COP23','COP22','COP21','COP20','COP19']
cop_df = {}

for cop_dir in cop_directories:
    directory = f'HLS_data\\{cop_dir}'
    files = os.listdir(directory)
    cop_df[cop_dir] = pd.DataFrame(files, columns=[cop_dir])

In [ ]:
# Print first five rows of COP28 dataframe
cop_df['COP28'].head()

In [ ]:
# Calculate the size of each DataFrame and store it in a dictionary
cop_df_size = {}
for cop_dir, df in cop_df.items():
    cop_df_size[cop_dir] = df.shape

print("Size of DataFrames:")
for cop_dir, size in cop_df_size.items():
    print(f"{cop_dir}: {size}")

In [ ]:
total_length = sum(len(df) for df in cop_df.values())
print("Total length of all DataFrames:", total_length)

In total, there are 1077 speeches over 10 different COPs.
COP22 has the most speeches. 

Next step: evaluate availability per country and in what languages. 

#### Combine all into a single dataframe

In [ ]:
HLS_df = pd.concat(cop_df.values(), axis=1)
HLS_df

#### List of parties
List of parties in the unfccc [link](https://unfccc.int/process/parties-non-party-stakeholders/parties-convention-and-observer-states) (Accessed 23-3)

In [ ]:
# Extract list of parties
party_list = pd.read_excel('HLS_data/party_list.xlsx')
# Set party names to be lowercase
party_list['Party'] = party_list['Party'].str.lower()

HLS_df = pd.concat([HLS_df,party_list], axis=1)
HLS_df

In [ ]:
# Save dataframe to excel
HLS_df.to_excel('HLS_overview.xlsx', index=False)

#### Original idea to merge party names with documents 
Merge party names with document names. If there is no match, enter a nan.value

Differing spellings make it quicker to do final manipulation with excel 
Examples: Use of EU vs European Union, Switserland vs Switzerland, st kitts vs Saint kitts

Party list only contains only spelling available on UNFCCC website. 

Solution: manual evaluation

In [ ]:
# Function to extract country names from document names
def extract_party_name(doc_name):
    for party in party_list['Party']:
        if party in doc_name:
            return party
    return None

In [ ]:
# Create separate dataframes per COP
def create_df(directory):
    files = os.listdir(directory)
    return pd.DataFrame(files, columns=['Document Names'])

COP28 = create_df('HLS_data\COP28')
COP27 = create_df('HLS_data\COP27')
COP26 = create_df('HLS_data\COP26')
COP25 = create_df('HLS_data\COP25')
COP24 = create_df('HLS_data\COP24')
COP23 = create_df('HLS_data\COP23')
COP22 = create_df('HLS_data\COP22')
COP21 = create_df('HLS_data\COP21')
COP20 = create_df('HLS_data\COP20')
COP19 = create_df('HLS_data\COP19')

In [ ]:
# Make all letters in document names lowercase (problem for COP28-COP24)
COP28['Document Names'] = COP28['Document Names'].str.lower() 
COP27['Document Names'] = COP27['Document Names'].str.lower() 
COP26['Document Names'] = COP26['Document Names'].str.lower() 
COP25['Document Names'] = COP25['Document Names'].str.lower() 
COP24['Document Names'] = COP24['Document Names'].str.lower() 

In [ ]:
# Apply the function to create a new column with extracted country names
COP28['Party'] = COP28['Document Names'].apply(extract_party_name)

# Print new dataframe
COP28.head()

In [ ]:
# Print rows with none values
COP28_na = COP28[COP28.Party.isna()]
COP28_na

In [ ]:
# Merge into one dataframe
merged_df= pd.merge(party_list, COP28, on='Party', how='outer')
merged_df.rename(columns={'Document Names':'HLS_COP28'}, inplace=True)
merged_df.to_excel('COP28_overview.xlsx', index=False)

#### Apply same idea for all HLS speeches

In [ ]:
cop_dfs = {'COP28': COP28, 'COP27': COP27, 'COP26': COP26, 'COP25': COP25, 'COP24': COP24, 
           'COP23': COP23, 'COP22': COP22, 'COP21': COP21, 'COP20': COP20, 'COP19': COP19}

for cop_name, cop_df in cop_dfs.items():
    cop_df['Party'] = cop_df['Document Names'].apply(extract_party_name)
    cop_df.rename(columns={'Document Names': cop_name}, inplace=True)


In [ ]:
# Merge each HLS dataframe with party list
COP27 = pd.merge(party_list, COP27,on='Party', how='outer')
COP26 = pd.merge(party_list, COP26, on='Party', how='outer')
COP25 = pd.merge(party_list, COP25, on='Party', how='outer')
COP24 = pd.merge(party_list, COP24, on='Party', how='outer')
COP23 = pd.merge(party_list, COP23, on='Party', how='outer')
COP22 = pd.merge(party_list, COP22, on='Party', how='outer')
COP21 = pd.merge(party_list, COP21, on='Party', how='outer')
COP20 = pd.merge(party_list, COP20, on='Party', how='outer')
COP19 = pd.merge(party_list, COP19, on='Party', how='outer')

In [ ]:
COP27.to_excel('COP27_overview.xlsx', index=False)
COP26.to_excel('COP26_overview.xlsx', index=False)
COP25.to_excel('COP25_overview.xlsx', index=False)
COP24.to_excel('COP24_overview.xlsx', index=False)
COP23.to_excel('COP23_overview.xlsx', index=False)
COP22.to_excel('COP22_overview.xlsx', index=False)
COP21.to_excel('COP21_overview.xlsx', index=False)
COP20.to_excel('COP20_overview.xlsx', index=False)
COP19.to_excel('COP19_overview.xlsx', index=False)

#### Work with COP_HLS dataframe
Check amount of missing speeches
Check number of speeches per country

IMPORTANT 
COP26 includes 2 more EU speeches. Some countries were labelled double,  due to multiple languages availabe and scraped. 

In total there are 10 COPs taken into account. 

In [ ]:
# Open new sorted dataframe
COP_HLS = pd.read_excel('COP_HLS.xlsx')
COP_HLS.drop(COP_HLS.index[198:203], inplace=True)
COP_HLS.set_index(COP_HLS.columns[0], inplace=True)
COP_HLS

In [ ]:
# Gain insight in Nan Values per COP
COP_HLS.isna().sum()

### Make new dataframe to evaluate vailability per country
1: convert to numbers

In [ ]:
# Replace all Na with 0
COP_HLS.fillna(0)

# Define a function to replace text entries with 1
def replace_text_with_1(value):
    if isinstance(value, str):  # Check if the value is a string
        return 1
    else:
        return value

# Apply the function to the entire DataFrame
HLS_num = COP_HLS.applymap(replace_text_with_1)

HLS_num

In [ ]:
# Add column with total count per COP
HLS_num['Total'] = HLS_num.sum(axis=1)

HLS_num

In [ ]:
# NR of available speeches per COP
HLS_num.count()

In [ ]:
HLS_num.to_excel('HLS_num.xlsx')

### Beschikbaarheid per land - basic evaluatie
17 landen hebben minder dan 3 speeches beschikbaar.
Alleen de Cook Islands hebben geen speeches beschikbaar. 

In [ ]:
#Countries that have less than 3 speeches available
lim_av = HLS_num[HLS_num['Total'] < 3].index
lim_av

17 landen hebben minder dan 3 speeches beschikbaar

In [ ]:
HLS_num[HLS_num['Total'] == 1].index

In [ ]:
HLS_num[HLS_num['Total'] == 0].index

In [ ]:
column_average = HLS_num['Total'].mean()
column_min = HLS_num['Total'].min()
column_max = HLS_num['Total'].max()

print("Average number of speeches available:", column_average)
print("Minimum number of speeches available:", column_min)
print("Maximum number of speeches available:", column_max)

### Manual check for languages
New dataframe, quick analysis.

Initial language evaluation presents 13 countries without an english speech available. Total number of available english written speeches: 822

In [3]:
HLS_eng = pd.read_excel('COP_HLS_language.xlsx')
HLS_eng.drop(HLS_eng.index[198:203], inplace=True)
HLS_eng.set_index(HLS_eng.columns[0], inplace=True)
HLS_eng

,COP28,COP27,COP26,COP25,COP24,COP23,COP22,COP21,COP20,COP19
Party,,,,,,,,,,
Afghanistan,NaN,NaN,NaN,afghanistan_cop25cmp15cma2_hls_en.pdf,NaN,afghanistan_cop23cmp13cma1-2_hls.pdf,afghanistan_cop22cmp12cma1_hls.pdf,cop21cmp11_hls_speech_afghanistan.pdf,cop20_hls_afghanistan.pdf,cop19_hls_afghanistan.pdf
Albania,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cop20_hls_albania.pdf,NaN
Algeria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cop21cmp11_hls_speech_algeria.pdf,NaN,NaN
Andorra,NaN,NaN,NaN,NaN,NaN,NaN,andorra_cop22cmp12cma1_hls.pdf,NaN,NaN,NaN
Angola,angola_cop28cmp18cma5_hls_eng.pdf,angola_cop27cmp17cma4_hls_eng.pdf,angola_cop26cmp16cma3_hls_en.pdf,NaN,NaN,angola_cop23cmp13cma1-2_hls.pdf,NaN,cop21cmp11_hls_speech_angola.pdf,cop20_hls_angola_english.pdf,cop19_hls_angola.pdf
...,...,...,...,...,...,...,...,...,...,...
Venezuela (Bolivarian Republic of),NaN,NaN,NaN,NaN,NaN,NaN,NaN,cop21cmp11_hls_speech_venezuela.pdf,cop20_hls_venezuela.pdf,NaN
Viet Nam,viet_nam_cop28cmp18cma5_hls_eng.pdf,NaN,viet_nam_cop26cmp16cma3_hls_en.pdf,NaN,NaN,NaN,vietnam_cop22cmp12cma1_hls.pdf,NaN,NaN,cop19_hls_vietnam.pdf
Yemen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Replace all Na with 0
HLS_eng.fillna(0)

# Define a function to replace text entries with 1
def replace_text_with_1(value):
    if isinstance(value, str):  # Check if the value is a string
        return 1
    else:
        return value

# Apply the function to the entire DataFrame
HLS_eng_num = HLS_eng.applymap(replace_text_with_1)
# Add column with total count per COP
HLS_eng_num['Total'] = HLS_eng_num.sum(axis=1)

HLS_eng_num

,COP28,COP27,COP26,COP25,COP24,COP23,COP22,COP21,COP20,COP19,Total
Party,,,,,,,,,,,
Afghanistan,NaN,NaN,NaN,1.0,NaN,1.0,1.0,1.0,1.0,1.0,6.0
Albania,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0
Algeria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0
Andorra,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0
Angola,1.0,1.0,1.0,NaN,NaN,1.0,NaN,1.0,1.0,1.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...
Venezuela (Bolivarian Republic of),NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,2.0
Viet Nam,1.0,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,1.0,4.0
Yemen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [5]:
#Countries that have less than 3 speeches available
lim_av = HLS_eng_num[HLS_eng_num['Total'] < 3].index
lim_av

Index(['Albania', 'Algeria', 'Andorra', 'Argentina', 'Bahrain', 'Benin',
       'Bolivia (Plurinational State of)', 'Botswana', 'Bulgaria',
       'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cameroon',
       'Central African Republic', 'Chad', 'Chile', 'Colombia', 'Comoros',
       'Congo', 'Cook Islands', 'Côte d'Ivoire', 'Dominican Republic',
       'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Fiji',
       'France', 'Gabon', 'Guinea', 'Haiti', 'Iraq', 'Lebanon', 'Libya',
       'Madagascar', 'Mali', 'Mauritania', 'Mexico', 'Nicaragua', 'Oman',
       'Panama', 'Paraguay', 'Peru', 'Qatar', 'Republic of Moldova',
       'Sao Tome and Principe', 'Senegal', 'Spain', 'Sudan', 'Switzerland',
       'Syrian Arab Republic', 'Togo', 'Trinidad and Tobago', 'Tunisia',
       'Turkmenistan', 'United Kingdom of Great Britain and Northern Ireland',
       'Uruguay', 'Uzbekistan', 'Venezuela (Bolivarian Republic of)', 'Yemen'],
      dtype='object', name='Party')

In [6]:
HLS_eng_num[HLS_eng_num['Total'] == 0].index

Index(['Argentina', 'Cameroon', 'Central African Republic', 'Chile',
       'Colombia', 'Congo', 'Cook Islands', 'Côte d'Ivoire',
       'Equatorial Guinea', 'Mauritania', 'Sao Tome and Principe', 'Tunisia',
       'Yemen'],
      dtype='object', name='Party')

In [8]:
HLS_eng_num.sum()

COP28     79.0
COP27     72.0
COP26     94.0
COP25     66.0
COP24     48.0
COP23     94.0
COP22    102.0
COP21    103.0
COP20     75.0
COP19     89.0
Total    822.0
dtype: float64

In [ ]:
column_average = HLS_eng_num['Total'].mean()
column_min = HLS_eng_num['Total'].min()
column_max = HLS_eng_num['Total'].max()

print("Average number of speeches available:", column_average)
print("Minimum number of speeches available:", column_min)
print("Maximum number of speeches available:", column_max)

In [ ]:
# Nr of HLS available per country in ENG
HLS_eng_num.count()

### (EXTRA) Do something with visualisation - world map
already have a cleaned file with geometry connected to country names

In [ ]:
gdf.head()

In [ ]:
gdf_stripped = gdf.drop(columns=['pop_est','continent','iso_a3','gdp_md_est'])
gdf_stripped.rename(columns={'name': 'Party'}, inplace=True)
gdf_stripped